# Notebook that downloads GPM rainfall data done per typhoon

In [191]:
%load_ext jupyter_black
# libraries
import pandas as pd
import numpy as np
import os
import datetime as dt
from pathlib import Path
from bs4 import BeautifulSoup
import requests
import getpass

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [192]:
# Setting directory
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/03_rainfall/input"
)
# Importing local libraries
typhoon_metadata = pd.read_csv(input_dir / "metadata_typhoons.csv")
# Setting path to save the GPM data
gpm_file_name = "gpm_data/rainfall_data/output_hhr/"
gpm_folder_path = Path(input_dir, gpm_file_name)

In [193]:
# To create an account for downloading the data
# follow the instructions here: https://registration.pps.eosdis.nasa.gov/registration/
# Change the user name and provide the password in the code
user_name = "pauline.ndirangu@un.org"
password = getpass.getpass(prompt="Password: ", stream=None)
baseurl = "https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata"
# y=2021
# m=4
# d=13
# file=f"/{y}/0{m}/{d}/gis"
# url=baseurl+file
ext = "tif"
##########################
# Setting the number of days prior to the landfall data for which to collect data
days_to_landfall = 2

# Default = FALSE
# IMPORTANT: setting to TRUE means that
# all downloaded GPM files will be deleted and re-downloaded
delete_folders = False

In [194]:
#%% Functions used
def date_range(start_date, end_date):
    return [
        str(start_date + dt.timedelta(days=x))
        for x in range((end_date - start_date).days + 1)
    ]


def list_files(url, user_name, password, ext=""):
    try:
        page = requests.get(url, auth=(user_name, password)).text
        soup = BeautifulSoup(page, "html.parser")
    except requests.exceptions.RequestException as error:
        print("Error:\n", error)
    return [
        url + "/" + node.get("href")
        for node in soup.find_all("a")
        if node.get("href").endswith(ext)
    ]


def download_gpm_http(start_date, end_date, download_path, type_imerg):
    # Username and password for logging in
    # Can create own account on NASA site
    # user_name = "pauline.ndirangu@un.org"#GPM_USERNAME
    base_url = ""

    # Connection to the site, if pasting in chrome:
    # https://arthurhouhttps.pps.eosdis.nasa.gov/
    # Directory to where the data is saved
    if type_imerg == "final":
        base_url = baseurl

    date_list = date_range(start_date, end_date)
    file_list = []

    for date in date_list:
        print(date)
        date_obj = dt.datetime.strptime(date, "%Y-%m-%d")
        d, m, y = (
            "%02d" % date_obj.day,
            "%02d" % date_obj.month,
            "%04d" % date_obj.year,
        )
        day_path = download_path / str(y + m + d)

        # Make a folder for each day, to save GPM data

        os.makedirs(day_path, exist_ok=True)
        if type_imerg != "final":
            continue

        data_dir_final = "/" + date_obj.strftime("%Y/%m/%d") + "/gis"
        url = base_url + data_dir_final
        tiff_files = list_files(
            url, user_name=user_name, password=user_name, ext="tif"
        )

        for tifffile in tiff_files:
            file_name = tifffile.split("/")[-1]

            file_path = day_path / file_name
            file_list.append(file_path)
            try:
                r = requests.get(tifffile, auth=(user_name, user_name))
                open(file_path, "wb").write(r.content)
            except requests.exceptions.RequestException as error:
                print("Error:\n", error)

    return file_list

In [196]:
# To make sure the dates can be converted to date type
typhoon_metadata["startdate"] = [
    str_col.replace("/", "-") for str_col in typhoon_metadata["startdate"]
]
typhoon_metadata["enddate"] = [
    str_col.replace("/", "-") for str_col in typhoon_metadata["enddate"]
]
typhoon_metadata["landfalldate"] = [
    str_col.replace("/", "-") for str_col in typhoon_metadata["landfalldate"]
]

In [126]:
# Typhoons for which to download rainfall data
typhoons = list(typhoon_metadata.typhoon.values)

# if there is already rainfall data in the project folder  output_hhr_processed
typhoons_with_rainfall_data = [
    items.split("_")[0]
    for items in os.listdir(str(input_dir) + "/" + gpm_file_name)
]
typhoons_without_rainfall_data = [
    items for items in typhoons if items not in typhoons_with_rainfall_data
]

In [127]:
typhoon_metadata_ = typhoon_metadata.set_index("typhoon").to_dict()
typhoons_dict = dict()

###################################################################
### START OF PROCESSING
###################################################################

for typhoon in typhoons_without_rainfall_data:
    case = typhoon
    typhoons_dict[case] = {
        "typhoon": typhoon,
        "dates": [
            dt.datetime.strptime(
                typhoon_metadata_["landfalldate"][typhoon], "%d-%m-%Y"
            ).date()
            - dt.timedelta(days=2),
            dt.datetime.strptime(
                typhoon_metadata_["landfalldate"][typhoon], "%d-%m-%Y"
            ).date()
            + dt.timedelta(days=2),
            dt.datetime.strptime(
                typhoon_metadata_["landfalldate"][typhoon], "%d-%m-%Y"
            ).date(),
        ],
        "imerg_type": typhoon_metadata_["imerg_type"][typhoon],
    }

This section takes a long time to download the data. 

In [ ]:
for keys in typhoons_dict.keys():
    typhoon_to_process = typhoons_dict[keys]
    typhoon = typhoon_to_process.get("typhoon")

    # Start/End date for precipitation data, get from the dictionary
    start_date = min(typhoon_to_process.get("dates"))
    end_date = max(typhoon_to_process.get("dates"))
    print("start_date is:", start_date, "end date of typhoon is:", end_date)

    # IMERG data type, either "early" (6hr), "late" (18hr) or "final" (4 months),
    # see https://pps.gsfc.nasa.gov/Documents/README.GIS.pdf
    imerg_type = typhoon_to_process.get("imerg_type")  # "early"
    print("imerg_type:", imerg_type)

    t0 = dt.datetime.now()

    # Specify the names to save the GPM data (folder) and the output file
    subfolder = typhoon  # + "/"
    gpm_path = gpm_folder_path / subfolder / "GPM"

    # Downloading rainfall data
    if not imerg_type == "trmm":
        download_gpm_http(start_date, end_date, gpm_path, imerg_type)

In [177]:
data_dir_final = "/" + date_obj.strftime("%Y/%m/%d") + "/gis"
base_url = baseurl
url = base_url + data_dir_final
tiff_files = list_files(
    url, user_name=user_name, password=user_name, ext="tif"
)
tiff_files

['https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata/2006/07/28/gis/3B-DAY-GIS.MS.MRG.3IMERG.20060728-S000000-E235959.6240.V06B.tif',
 'https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata/2006/07/28/gis/3B-HHR-GIS.MS.MRG.3IMERG.20060728-S000000-E002959.0000.V06B.tif',
 'https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata/2006/07/28/gis/3B-HHR-GIS.MS.MRG.3IMERG.20060728-S003000-E005959.0030.V06B.tif',
 'https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata/2006/07/28/gis/3B-HHR-GIS.MS.MRG.3IMERG.20060728-S010000-E012959.0060.V06B.tif',
 'https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata/2006/07/28/gis/3B-HHR-GIS.MS.MRG.3IMERG.20060728-S013000-E015959.0090.V06B.tif',
 'https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata/2006/07/28/gis/3B-HHR-GIS.MS.MRG.3IMERG.20060728-S020000-E022959.0120.V06B.tif',
 'https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata/2006/07/28/gis/3B-HHR-GIS.MS.MRG.3IMERG.20060728-S023000-E025959.0150.V06B.tif',
 'https://arthurhouhttps.pps.eosdi